In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

2024-03-07 06:39:12.736725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 06:39:12.736853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 06:39:12.883123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

DIM = (176,176)

work_dr = IDG()

train_ds = train_datagen.flow_from_directory(
        "/kaggle/input/knee-data-oa/Dataset/Cropped/train",
        seed=123,
        target_size=(176, 176),
        batch_size=64,
        class_mode='sparse'
        )

test_ds = test_datagen.flow_from_directory(
"/kaggle/input/knee-data-oa/Dataset/Cropped/test",
seed=123,
target_size=(176, 176),
batch_size=64,
    class_mode='sparse'
)

val_ds = val_datagen.flow_from_directory(
"/kaggle/input/knee-data-oa/Dataset/Cropped/val",
seed=123,
target_size=(176, 176),
batch_size=64,
class_mode='sparse'
)

Found 5778 images belonging to 5 classes.
Found 1656 images belonging to 5 classes.
Found 826 images belonging to 5 classes.


In [3]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=10, 
                               restore_best_weights=True)

In [11]:
tf.cast(0.999999e-05 * tf.math.exp(-0.1), tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0483645e-06>

In [33]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return tf.cast(lr * tf.math.exp(-0.1), tf.float32)

lr_schedule = LearningRateScheduler(lr_scheduler)

### DenseNet169

*Thomas, Kevin A., et al. "Automated classification of radiographic knee osteoarthritis severity using deep neural networks." Radiology: Artificial Intelligence 2.2 (2020): e19006*

In [13]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

denseNet169 = DenseNet169(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
denseNet169.trainable = True
denseNet169_model_new = models.Sequential()
denseNet169_model_new.add(denseNet169)
denseNet169_model_new.add(layers.GlobalAveragePooling2D())
denseNet169_model_new.add(layers.Dense(128, activation='relu',  kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
denseNet169_model_new.add(layers.Dropout(0.3))
denseNet169_model_new.add(layers.Dense(5, activation='softmax'))

In [14]:
denseNet169_model_new.compile(optimizer=optimizers.Adam(learning_rate=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [15]:
denseNet169_new = denseNet169_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 365s 2s/step - accuracy: 0.3626 - loss: 3.9634 - val_accuracy: 0.4128 - val_loss: 3.3712 - learning_rate: 1.0000e-04
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 104s 625ms/step - accuracy: 0.4937 - loss: 3.1299 - val_accuracy: 0.3971 - val_loss: 2.8985 - learning_rate: 1.0000e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 61s 620ms/step - accuracy: 0.5469 - loss: 2.6562 - val_accuracy: 0.5230 - val_loss: 2.4804 - learning_rate: 1.0000e-04
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 61s 618ms/step - accuracy: 0.5787 - loss: 2.3234 - val_accuracy: 0.5521 - val_loss: 2.2223 - learning_rate: 1.0000e-04
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 61s 620ms/step - accuracy: 0.5832 - loss: 2.0857 - val_accuracy: 0.5690 - val_loss: 2.0175 - learning_rate: 1.0000e-04
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 61s 624ms/step - accuracy: 0.6014 - loss: 1.9200 - val_accuracy: 0.6174 - val_loss: 1.8263 - learning_rate: 1.0000e-04
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 62s 633ms/step - accu

In [16]:
loss, accuracy = denseNet169_model_new.evaluate(test_ds)

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.3860 - loss: 3.3880 


## VGG16

*Alshamrani, Hassan A., et al. "Osteo-NeT: An Automated System for Predicting Knee Osteoarthritis from X-ray Images Using Transfer-Learning-Based Neural Networks Approach." Healthcare. Vol. 11. No. 9. MDPI, 2023.*

In [34]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
vgg16.trainable = True
vgg16_model_new = models.Sequential()
vgg16_model_new.add(vgg16)
vgg16_model_new.add(layers.GlobalAveragePooling2D())
vgg16_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
vgg16_model_new.add(layers.Dropout(0.3))
vgg16_model_new.add(layers.Dense(5, activation='softmax'))  

In [35]:
vgg16_model_new.compile(optimizer=optimizers.Adam(learning_rate=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [36]:
vgg16_new = vgg16_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 68s 624ms/step - accuracy: 0.3567 - loss: 3.9465 - val_accuracy: 0.4031 - val_loss: 3.4735 - learning_rate: 1.0000e-04
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 601ms/step - accuracy: 0.3798 - loss: 3.6357 - val_accuracy: 0.3971 - val_loss: 3.0715 - learning_rate: 1.0000e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 58s 585ms/step - accuracy: 0.3824 - loss: 3.0053 - val_accuracy: 0.3971 - val_loss: 2.7493 - learning_rate: 1.0000e-04
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 57s 581ms/step - accuracy: 0.3881 - loss: 2.7422 - val_accuracy: 0.4479 - val_loss: 2.3291 - learning_rate: 1.0000e-04
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 57s 580ms/step - accuracy: 0.4293 - loss: 2.8796 - val_accuracy: 0.5242 - val_loss: 2.0177 - learning_rate: 1.0000e-04
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 82s 584ms/step - accuracy: 0.4859 - loss: 2.0820 - val_accuracy: 0.5230 - val_loss: 1.8597 - learning_rate: 1.0000e-04
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 595ms/step - acc

In [37]:
loss, accuracy = vgg16_model_new.evaluate(test_ds)

26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 210ms/step - accuracy: 0.3750 - loss: 3.4973


## ResNet101

*Mohammed, Abdul Sami, et al. "Knee Osteoarthritis Detection and Severity Classification Using Residual Neural Networks on Preprocessed X-ray Images." Diagnostics 13.8 (2023): 1380.*

In [38]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet101 = ResNet101(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet101.trainable = True
resnet101_model_new = models.Sequential()
resnet101_model_new.add(resnet101)
resnet101_model_new.add(layers.GlobalAveragePooling2D())
resnet101_model_new.add(layers.Dense(128, activation='relu',kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet101_model_new.add(layers.Dropout(0.3))
resnet101_model_new.add(layers.Dense(5, activation='softmax')) 

In [39]:
resnet101_model_new.compile(optimizer=optimizers.Adam(learning_rate=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [40]:
resnet101_new = resnet101_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50


W0000 00:00:1709798724.227694     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3775 - loss: 3.9836

W0000 00:00:1709798830.552651     127 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


91/91 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.3781 - loss: 3.9816 - val_accuracy: 0.3971 - val_loss: 3.9239 - learning_rate: 1.0000e-04
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 601ms/step - accuracy: 0.5082 - loss: 3.4235 - val_accuracy: 0.3971 - val_loss: 3.5360 - learning_rate: 1.0000e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 596ms/step - accuracy: 0.5799 - loss: 3.0319 - val_accuracy: 0.3971 - val_loss: 3.3494 - learning_rate: 1.0000e-04
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 596ms/step - accuracy: 0.5944 - loss: 2.7711 - val_accuracy: 0.3971 - val_loss: 3.3809 - learning_rate: 1.0000e-04
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 60s 606ms/step - accuracy: 0.6144 - loss: 2.5256 - val_accuracy: 0.3983 - val_loss: 3.1497 - learning_rate: 1.0000e-04
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 602ms/step - accuracy: 0.6383 - loss: 2.2605 - val_accuracy: 0.4092 - val_loss: 2.9997 - learning_rate: 1.0000e-04
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 58s 594ms/step - accuracy: 0.6364

In [41]:
loss, accuracy = resnet101_model_new.evaluate(test_ds)

26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 425ms/step - accuracy: 0.3764 - loss: 3.9441


### VGG19

*Chen, Pingjun, et al. "Fully automatic knee osteoarthritis severity grading using deep neural networks with a novel ordinal loss." Computerized Medical Imaging and Graphics 75 (2019): 84-92.*

In [42]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
vgg19.trainable = True
vgg19_model_new = models.Sequential()
vgg19_model_new.add(vgg19)
vgg19_model_new.add(layers.GlobalAveragePooling2D())
vgg19_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
vgg19_model_new.add(layers.Dropout(0.3))
vgg19_model_new.add(layers.Dense(5, activation='softmax')) 

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [44]:
vgg19_model_new.compile(optimizer=optimizers.Adam(learning_rate=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [45]:
vgg19_new = vgg19_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 73s 657ms/step - accuracy: 0.3520 - loss: 3.9003 - val_accuracy: 0.3971 - val_loss: 3.4834 - learning_rate: 1.0000e-04
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 60s 612ms/step - accuracy: 0.3863 - loss: 3.3955 - val_accuracy: 0.3971 - val_loss: 3.1153 - learning_rate: 1.0000e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 60s 609ms/step - accuracy: 0.3837 - loss: 3.0376 - val_accuracy: 0.4431 - val_loss: 2.7641 - learning_rate: 1.0000e-04
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 599ms/step - accuracy: 0.3962 - loss: 2.9014 - val_accuracy: 0.4019 - val_loss: 2.5200 - learning_rate: 1.0000e-04
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 58s 595ms/step - accuracy: 0.4231 - loss: 2.4673 - val_accuracy: 0.5048 - val_loss: 2.1858 - learning_rate: 1.0000e-04
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 58s 589ms/step - accuracy: 0.4604 - loss: 2.3027 - val_accuracy: 0.4685 - val_loss: 2.0548 - learning_rate: 1.0000e-04
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 59s 604ms/step - acc

In [46]:
loss, accuracy = vgg19_model_new.evaluate(test_ds)

26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 159ms/step - accuracy: 0.3795 - loss: 3.4813
